# Supervisor Multi-Agent Architecture

This notebook is a simple implementation of a supervisor agent architecture with two ReAct agents as subagents.

In [7]:
# Import libraries
from pydantic import BaseModel, Field
from typing import List, Optional, Annotated, Literal

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

from langgraph.graph import StateGraph, START, END, MessagesState

from codes.config.config import config

In [8]:
# Define schemas
class SupervisorResponse(BaseModel):
    route: Literal['math_agent', 'search_agent', 'end']

class GraphState(MessagesState):
    super_visor_route: Optional[SupervisorResponse]
    math_result: Optional[str]
    search_result: Optional[str]

In [9]:
# Create a ChatModel
llm = ChatOpenAI(api_key=config.openai_api_key.get_secret_value(), model="gpt-4o")

In [ ]:
# Supervisor Node
system_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""You are a helpful assistant. You are designed to give accurate answers to the users.
Your main goal is to provide accurate answers to the user and NEVER guess the correct answer based on your assumptions.

You have two specialist agents working for you to make sure you always provide accurate answers, here is their description:
1. math_agent: It handles calculations, equations, and mathematical problems.
2. search_agent: It handles information retrieval, current events, and factual queries.

When you need one of them, or you finished generating user question, call the tool you have.
"""
)

def supervisor(state: GraphState):
    """Supervisor node to route the conversation to the appropriate agent or craft the final response."""
    messages = state["messages"]
